In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from collections import defaultdict

In [2]:
def read_data(filename):
    validation_traj = np.load(filename, allow_pickle=True)
    valid_data = validation_traj.item()
    pose_set = set()
    for user_name in valid_data.keys():
        for path_name in valid_data[user_name].keys():
            for posture_name in valid_data[user_name][path_name].keys():
                pose_set.add(posture_name)
    posture_data = defaultdict(list)

    for user_name in valid_data.keys():
        for path_name in valid_data[user_name].keys():
            for posture_name in valid_data[user_name][path_name].keys():
                data = valid_data[user_name][path_name][posture_name]
                pos = (data[:, [0, 1]] + np.array((1, 1))) * 0.6
                mag = data[:, [3, 4, 5, 6, 7, 8, 12, 13, 14]]
                gravity_direction = estimate_gravity(mag[:, 0:3], mag[:, 3:6])
                aligned_magnetometer_data = align_magnetometer_with_gravity(mag[:, 6:9], gravity_direction)
                pos_mag = np.concatenate((pos, aligned_magnetometer_data), axis=-1)
                posture_data[posture_name].append(pos_mag)
    
    return posture_data

In [3]:
def split_traj(trajs, length=20):
    sp_traj = []
    for traj in trajs:
        for i in range(len(traj) - length + 1):
            sp_traj.append(traj[i:i+length])
    return np.array(sp_traj)

In [4]:
def estimate_gravity(gravity_included_acceleration, dynamic_acceleration):
    raw_gravity_vector = np.array(gravity_included_acceleration) - np.array(dynamic_acceleration)
    gravity_direction = raw_gravity_vector / np.linalg.norm(raw_gravity_vector)
    return gravity_direction

In [5]:
def align_magnetometer_with_gravity(magnetometer_data, gravity_direction):
    target_gravity_direction = np.array([0, 0, 1])
    rotation_vector = np.cross(gravity_direction, target_gravity_direction)
    rotation_vector_magnitude = np.linalg.norm(rotation_vector)
    if rotation_vector_magnitude == 0:
        return magnetometer_data  # 如果已经对齐，则不需要旋转
    rotation_vector /= rotation_vector_magnitude
    angle = np.arccos(np.dot(gravity_direction, target_gravity_direction))
    rotation = R.from_rotvec(rotation_vector * angle[:, np.newaxis])
    rotated_magnetometer_data = rotation.apply(magnetometer_data)
    return rotated_magnetometer_data

In [6]:
filename = 'all_user_match_data.npy'
posture_data = read_data(filename)
for key, value in posture_data.items():
    trajs = split_traj(value)
    np.save(f'./baseline_npy/{key}.npy', trajs)
    print(trajs.shape)

(674, 20, 5)
(674, 20, 5)
(674, 20, 5)
(674, 20, 5)
(674, 20, 5)
